In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

import desc
from desc.objectives import *
from desc.basis import ZernikePolynomial, FourierZernikeBasis, DoubleFourierSeries
from desc.transform import Transform
from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.plotting import (
    plot_1d,
    plot_2d,
    plot_3d,
    plot_comparison, 
    plot_section, 
    plot_fsa, 
    plot_surfaces,
)
from desc.optimize import Optimizer
from desc.perturbations import *
import numpy as np
np.set_printoptions(linewidth=np.inf, threshold=sys.maxsize)

from desc.geometry import (
    FourierRZToroidalSurface,  
    SplineXYZCurve, 
    FourierXYZCurve,
)
%matplotlib inline
from desc.utils import copy_coeffs
import matplotlib.pyplot as plt
from desc.examples import get
from desc.objectives.getters import (
    get_fixed_boundary_constraints, 
    maybe_add_self_consistency,
)
from desc.grid import LinearGrid, QuadratureGrid, Grid
import plotly.graph_objects as go
from desc.profiles import PowerSeriesProfile
from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.compute import data_index
from desc.coils import SplineXYZCoil
from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.objectives import (
    ObjectiveFunction,
    ForceBalance,
    get_fixed_boundary_constraints,
)
from desc.optimize import Optimizer
from desc.plotting import plot_1d, plot_section, plot_surfaces
from desc.profiles import PowerSeriesProfile
from desc.examples import get
from desc.grid import LinearGrid
import plotly.graph_objects as go

from scipy.linalg import qr_insert, qr

import jax.numpy as jnp
from jax.lax import fori_loop
from jax.lax import rsqrt
import jax

import functools
from desc.objectives.normalization import compute_scaling_factors

DESC version 0.12.1+390.gfc160a9a9,using JAX backend, jax version=0.4.31, jaxlib version=0.4.31, dtype=float64
Using device: CPU, with 10.95 GB available memory


In [2]:
eq = get("HELIOTRON")

In [3]:
from desc.objectives.utils import factorize_linear_constraints, combine_args
constraints = get_fixed_boundary_constraints(eq)
constraints = maybe_add_self_consistency(eq, constraints)
constraints = ObjectiveFunction(constraints)
objective = ObjectiveFunction(ForceBalance(eq))
constraints.build()
objective.build()
objective, constraints = combine_args(objective, constraints)
xp, A, b, Z, unfixed, project, recover = factorize_linear_constraints(objective, constraints)

Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed iota
Building objective: fixed sheet current
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
Building objective: force
Precomputing transforms
0.0
0.0
0.0
0.0
0.0
0.0
5.189260187289721e-15
1.570829369336555e-15
0.0
6.1371721738636655e-15
1.1589285893609364e-16
2.524513058901832e-15
6.055875176239623e-16
0.0


In [3]:
from desc.objectives.utils import factorize_linear_constraints, combine_args
constraints = get_fixed_boundary_constraints(eq)
constraints = maybe_add_self_consistency(eq, constraints)
constraints = ObjectiveFunction(constraints)
objective = ObjectiveFunction(ForceBalance(eq))
constraints.build()
objective.build()
objective, constraints = combine_args(objective, constraints)
xp, A, b, Z, unfixed, project, recover = factorize_linear_constraints(objective, constraints)

Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed iota
Building objective: fixed sheet current
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
Building objective: force
Precomputing transforms
0.0
0.0
0.0
0.0
0.0
0.0
2.104729836894458e-14
3.0004050830366466e-15
0.0
4.7014203614482855e-14
5.8765769286766e-17
6.172714617515281e-15
5.52007406467261e-16
0.0


In [3]:
from desc.objectives.utils import factorize_linear_constraints, combine_args
constraints = get_fixed_boundary_constraints(eq)
constraints = maybe_add_self_consistency(eq, constraints)
constraints = ObjectiveFunction(constraints)
objective = ObjectiveFunction(ForceBalance(eq))
constraints.build()
objective.build()
objective, constraints = combine_args(objective, constraints)
xp, A, b, Z, unfixed, project, recover = factorize_linear_constraints(objective, constraints)

Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed iota
Building objective: fixed sheet current
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
Building objective: force
Precomputing transforms
0.0
0.0
0.0
0.0
0.0
0.0
5.189260187289721e-15
1.570829369336555e-15
0.0
6.1371721738636655e-15
1.1589285893609364e-16
2.524513058901832e-15
6.055875176239623e-16
0.0


In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
im = ax.imshow(A, aspect="auto", interpolation='None', cmap='viridis')
fig.colorbar(im)
plt.savefig("A.png", dpi=1000)

plt.figure()
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
im = ax.imshow(Z, aspect="auto", interpolation='None', cmap='viridis')
fig.colorbar(im)
plt.savefig("Z.png", dpi=1000)

In [ ]:
# np.savetxt("poin-A.txt", A)
# np.savetxt("poin-Z.txt", Z)
# np.savetxt("poin-b.txt", b)
# np.savetxt("poin-xp.txt", xp)
# np.savetxt("poin-unfixed.txt", unfixed)

np.savetxt("master-A.txt", A)
np.savetxt("master-Z.txt", Z)
np.savetxt("master-b.txt", b)
np.savetxt("master-xp.txt", xp)
np.savetxt("master-unfixed.txt", unfixed)

In [3]:
A = np.loadtxt("master-A.txt")
Z = np.loadtxt("master-Z.txt")
b = np.loadtxt("master-b.txt")
xp = np.loadtxt("master-xp.txt")
unfixed = np.loadtxt("master-unfixed.txt")

A_poin = np.loadtxt("poin-A.txt")
Z_poin = np.loadtxt("poin-Z.txt")
b_poin = np.loadtxt("poin-b.txt")
xp_poin = np.loadtxt("poin-xp.txt")
unfixed_poin = np.loadtxt("poin-unfixed.txt")

In [9]:
print(f"{A.shape} {A_poin.shape}")
print(f"{Z.shape} {Z_poin.shape}")
print(f"{b.shape} {b_poin.shape}")
print(f"{xp.shape} {xp_poin.shape}")
print(f"{unfixed.shape} {unfixed_poin.shape}")

print(f"{(Z.T@xp[unfixed.astype(int)]).shape}")
print(f"{(Z_poin.T@xp_poin[unfixed_poin.astype(int)]).shape}")

x_reduced_m =  Z.T@xp[unfixed.astype(int)]
x_reduced_p = Z_poin.T@xp_poin[unfixed_poin.astype(int)]

assert np.allclose(x_reduced_m, x_reduced_p)
max(x_reduced_m - x_reduced_p)

(168, 1761) (413, 2006)
(1761, 1593) (2006, 1593)
(168,) (413,)
(1977,) (2296,)
(1761,) (2006,)
(1593,)
(1593,)


4.371503159461554e-16

In [ ]:
def find_indices_with_single_negative_one(A):
    for row, bi in zip(A, b_poin):
        prt = row[np.where(row!=0)]
        # prt[prt<0] = 0
        print(prt, bi)

find_indices_with_single_negative_one(A_poin)